## IPython notebook for the Fourier transform and low/high pass filtering.

### Import all modules and functions needed for this example.

In [ ]:
import numpy as np
from skimage.io import imread, imsave
from skimage import img_as_float
from skimage.exposure import rescale_intensity
import matplotlib.pyplot as plt


### Implementation of the low/high pass transfer function and the frequency filtering with a transfer function.

In [ ]:
def create_low_pass_kernel(shape, r):
    m, n = shape
    x, y = np.meshgrid(range(n), range(m))
    frequencies = np.square(x - m / 2) + np.square(y - n / 2)
    frequency_mask = frequencies < r**2
    return frequency_mask


def create_high_pass_kernel(shape, r1, r2):
    m, n = shape
    x, y = np.meshgrid(range(n), range(m))
    frequencies = np.square(x - m / 2) + np.square(y - n / 2)
    frequency_mask = np.logical_and(frequencies < r2**2, frequencies > r1**2)
    return frequency_mask


def filter_with_freq_kernel(input_image, kernel):
    # Apply FFT and shift so that the lowest frequency is at the central pixel.
    Ff = np.fft.fftshift(np.fft.fft2(input_image))
    # Multiply the image frequencies with the kernel
    Ff2 = np.multiply(kernel, Ff)
    # Shift back, transform back and discard the imaginary component.
    output_image = np.real(np.fft.ifft2(np.fft.ifftshift(Ff2)))
    return output_image


### Read and plot the input image.

In [ ]:
input_image = img_as_float(imread("3_8_12-GaN-Area2_29_HAADF_1000.png", as_gray=True))

plt.imshow(input_image, interpolation="nearest", cmap=plt.cm.get_cmap("gray"), vmin=0, vmax=1)
plt.axis("off")
plt.title("Original image")
plt.show()

### Plot the power spectrum of the image.

In [ ]:
# Compute the Fourier transform of the image
Ff = np.fft.fftshift(np.fft.fft2(input_image))
# The power spectrum is the pixel wise complex absolute value of the Fourier transform
power_spectrum = np.abs(Ff)

plt.subplot(1, 2, 1)
plt.imshow(power_spectrum, interpolation="nearest", cmap=plt.cm.get_cmap("gray"))
plt.axis("off")
plt.title("$T^{norm}(|FFT(f)|)$")
plt.subplot(1, 2, 2)
plt.imshow(np.log2(1 + power_spectrum), interpolation="nearest", cmap=plt.cm.get_cmap("gray"))
plt.axis("off")
plt.title("$T^{norm}(T^{log}(|FFT(f)|))$")
plt.show()

### Parameters for the low and high frequency filters.

In [ ]:
r = input_image.shape[0] // 10
r2 = 2 * r

### Create low and high frequency transfer functions.

In [ ]:
# Low pass frequency transfer function
low_pass_kernel_FT = create_low_pass_kernel(input_image.shape, r)
# Corresponding space domain kernel
low_pass_kernel = np.fft.ifftshift(np.fft.ifft2(low_pass_kernel_FT))

plt.suptitle("Low pass kernel", y=0.85)
plt.subplot(1, 2, 1)
plt.imshow(low_pass_kernel_FT, interpolation="nearest", cmap=plt.cm.get_cmap("gray"))
plt.axis("off")
plt.title("$FFT(\psi)$")
plt.subplot(1, 2, 2)
plt.imshow(np.log2(1 + np.abs(low_pass_kernel)), interpolation="nearest", cmap=plt.cm.get_cmap("gray"))
plt.axis("off")
plt.title("$T^{norm}(T^{log}(|\psi|))$")
plt.show()

In [ ]:
# High pass frequency transfer function
high_pass_kernel_FT = create_high_pass_kernel(input_image.shape, r, r2)
# Corresponding space domain kernel
high_pass_kernel = np.fft.ifftshift(np.fft.ifft2(high_pass_kernel_FT))

plt.suptitle("High pass kernel", y=0.85)
plt.subplot(1, 2, 1)
plt.imshow(high_pass_kernel_FT, interpolation="nearest", cmap=plt.cm.get_cmap("gray"))
plt.axis("off")
plt.title("$FFT(\psi)$")
plt.subplot(1, 2, 2)
plt.imshow(np.log2(1 + np.abs(high_pass_kernel)), interpolation="nearest", cmap=plt.cm.get_cmap("gray"))
plt.axis("off")
plt.title("$T^{norm}(T^{log}(|\psi|))$")
plt.show()

### Apply low and high pass filter.

In [ ]:
low_pass_filtered = filter_with_freq_kernel(input_image, low_pass_kernel_FT)
high_pass_filtered = filter_with_freq_kernel(input_image, high_pass_kernel_FT)

plt.subplot(1, 2, 1)
plt.imshow(low_pass_filtered, interpolation="nearest", cmap=plt.cm.get_cmap("gray"))
plt.axis("off")
plt.title("$FFT(\psi)$")
plt.subplot(1, 2, 2)
plt.imshow(high_pass_filtered, interpolation="nearest", cmap=plt.cm.get_cmap("gray"))
plt.axis("off")
plt.title("$T^{norm}(T^{log}(|\psi|))$")
plt.show()
